In [3]:
import os
import numpy as np
import matplotlib as mp
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import Normalize
from matplotlib.ticker import AutoMinorLocator
import matplotlib.cm as cm
import flopy
import flopy.utils.binaryfile as bf

print(f"FloPy version: {flopy.__version__}")

FloPy version: 3.10.0


In [4]:
# /workspaces/hwrs564b_course_materials_JessicaCarvalho007/Homework/zz_Practice_MODFLOW
# File paths
model_top_dir = "zz_Practice_MODFLOW"
input_dir = os.path.join(model_top_dir, "Model_Scenarios")
if not os.path.exists(input_dir):
    print("Creating workspace directory")
    os.makedirs(input_dir, exist_ok=True)

# exe_name ="/workspaces/modflow/mf2005"
exe_name = "../../../modflow/mf2005"         # MF2005 executable
assert os.path.exists(exe_name), f"mf2005 not found at: {exe_name}"

In [5]:
# DIS
Lx = 2500.0                 # meters
Ly = 2500.0                 # meters
dx = dy = 100.0              # cell width in meters
nrow = int(Ly / dy)         # Number of rows
ncol = int(Lx / dx)         # Number of columns

nlay = 3                    # Number of layers
ztop = 100                  # Top of the model domain (m)
zbot = 0.0                  # Bottom of the model domain (m)
dz   = (ztop - zbot) / nlay # Thickness of each layer (m)

nper   = 1                  # Number of stress periods
perlen = [1.0]              # Length of each stress period in days
nstp   = [1]                # Number of time steps in each stress period
steady = [True]             # Steady-state (True) or transient (False) stress period flag
itmuni = 4                  # Time unit (4 = days)
lenuni = 2                  # Length unit (2 = meters)

m   = flopy.modflow.Modflow(modelname=modelname, model_ws=model_ws, exe_name=exe_name, version="mf2005")

# For Grid
x_centers = np.array([(c+0.5)*dx for c in range(ncol)])
y_centers = np.array([Ly - (r+0.5)*dy for r in range(nrow)])
r_mid = nrow // 2           # Row index of the middle row
c_mid = ncol // 2           # Column index of the middle column

# LPF
laytyp = 1                  # 0 = confined, >0 = unconfined
n  = 0.35                   # porosity
S = 0.001                   # storage coefficient for confined layers (if laytyp=0) or specific storage for unconfined layers (if laytyp>0)
Sy = 0.30                   # specific yield for unconfined layers (if laytyp>0) else 0.30 * dz  ???
Ss = 0.001                  # specific storage for confined layers (if laytyp=0) else 0.001 / dz ???

# BAS
ibound = np.ones((nlay, nrow, ncol), dtype=np.int32)
ibound[:,:,0]  = -1         #replace leftmost column (all zs, all ys, first x) with -1 to indicate constant head 
ibound[:,:,-1] = -1         #replace leftmost column (all zs, all ys, first x) with -1 to indicate constant head 

NameError: name 'modelname' is not defined

In [ ]:

rch = flopy.modflow.ModflowRch(model=m, rech=rech, ipakcb=53)
wel = flopy.modflow.ModflowWel(model=m, stress_period_data={0: wel_data_sp0}) if wells else None
# pcg = flopy.modflow.ModflowPcg(model=m)
pcg = flopy.modflow.ModflowPcg(model=m)
oc  = flopy.modflow.ModflowOc(model=m, stress_period_data={(0,0):['print head', 'print budget', 'save head', 'save budget']}, compact=True)

m.write_input()

In [ ]:
# BAS
ibound = np.ones((nlay, nrow, ncol), dtype=np.int32)
ibound[:,:,0]  = -1         #replace leftmost column (all zs, all ys, first x) with -1 to indicate constant head 
ibound[:,:,-1] = -1         #replace leftmost column (all zs, all ys, first x) with -1 to indicate constant head 

# For a mean single H value everywhere
strt = np.full((nlay, nrow, ncol), 40, dtype=np.float32)
strt[0, :, 0] = 50
strt[0, :, -1] = 30
bas = flopy.modflow.ModflowBas(model=m, ibound=ibound, strt=strt)

In [ ]:
# For a mean single K value everywhere
hk = np.full((nlay, nrow, ncol), 5, dtype=np.float32)
hk[1, :, :] = 2
hk[2, :, :] = 3
lpf = flopy.modflow.ModflowLpf(model=m, laytyp=laytyp, hk=hk, vka=2, ss=Ss, sy=Sy, storagecoefficient=True, ipakcb=53)